# Multimodal Pipeline - Текст + Изображения

Этот notebook содержит пайплайны для:
- CLIP (сопоставление текста и изображений)
- Image Captioning
- Visual Question Answering
- Text-to-Image поиск

In [ ]:
!pip install transformers pillow torch torchvision clip-by-openai -q

In [ ]:
import torch
import clip
from PIL import Image
import pandas as pd
import numpy as np
from transformers import CLIPProcessor, CLIPModel, BlipProcessor, BlipForConditionalGeneration
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

## 1. CLIP - сопоставление текста и изображений

In [ ]:
# === ВАШИ НАСТРОЙКИ ===
IMAGE_DIR = './images'  # Папка с изображениями
# Создайте папку и поместите туда изображения

# Загрузка CLIP
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

print("✓ CLIP модель загружена!")

In [ ]:
# Функция для сопоставления изображения с текстами
def match_image_with_texts(image_path, texts):
    image = Image.open(image_path)
    
    inputs = processor(
        text=texts,
        images=image,
        return_tensors="pt",
        padding=True
    ).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits_per_image = outputs.logits_per_image
        probs = logits_per_image.softmax(dim=1)
    
    return probs.cpu().numpy()[0]

# === ПРИМЕР ИСПОЛЬЗОВАНИЯ ===
# texts = ["кошка", "собака", "автомобиль", "дерево"]
# image_path = Path(IMAGE_DIR) / "example.jpg"
# scores = match_image_with_texts(image_path, texts)
# for text, score in zip(texts, scores):
#     print(f"{text}: {score:.4f}")

In [ ]:
# Получение эмбеддингов для изображений и текстов
def get_image_embeddings(image_paths, batch_size=32):
    embeddings = []
    
    for i in range(0, len(image_paths), batch_size):
        batch_paths = image_paths[i:i+batch_size]
        images = [Image.open(p) for p in batch_paths]
        
        inputs = processor(images=images, return_tensors="pt").to(device)
        
        with torch.no_grad():
            image_features = model.get_image_features(**inputs)
            embeddings.append(image_features.cpu().numpy())
    
    return np.vstack(embeddings)

def get_text_embeddings(texts, batch_size=32):
    embeddings = []
    
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        
        inputs = processor(text=batch_texts, return_tensors="pt", padding=True).to(device)
        
        with torch.no_grad():
            text_features = model.get_text_features(**inputs)
            embeddings.append(text_features.cpu().numpy())
    
    return np.vstack(embeddings)

## 2. Image Captioning - описание изображений

In [ ]:
# Загрузка BLIP для генерации описаний
caption_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
caption_model = BlipForConditionalGeneration.from_pretrained(
    "Salesforce/blip-image-captioning-base"
).to(device)

print("✓ BLIP модель загружена!")

In [ ]:
def generate_caption(image_path):
    image = Image.open(image_path).convert('RGB')
    
    inputs = caption_processor(image, return_tensors="pt").to(device)
    
    with torch.no_grad():
        output = caption_model.generate(**inputs, max_length=50)
    
    caption = caption_processor.decode(output[0], skip_special_tokens=True)
    return caption

# === ПРИМЕР ===
# caption = generate_caption(Path(IMAGE_DIR) / "example.jpg")
# print(f"Описание: {caption}")

## 3. Предсказания для соревнования

In [ ]:
# === ЗАГРУЗИТЕ СВОИ ДАННЫЕ ===
# Пример: CSV с путями к изображениям и текстовыми метками
test_df = pd.read_csv('test.csv')

# Если задача - классификация изображений по текстовым меткам
# class_labels = ["класс1", "класс2", "класс3"]  # Ваши классы

# predictions = []
# for img_path in test_df['image_path']:
#     scores = match_image_with_texts(img_path, class_labels)
#     pred_class = class_labels[np.argmax(scores)]
#     predictions.append(pred_class)

# submission = pd.DataFrame({
#     'id': test_df['id'],
#     'prediction': predictions
# })
# submission.to_csv('multimodal_submission.csv', index=False)

## 4. Поиск изображений по текстовому запросу

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Получение эмбеддингов для всех изображений
# image_paths = list(Path(IMAGE_DIR).glob('*.jpg'))
# image_embeddings = get_image_embeddings(image_paths)

# Поиск по текстовому запросу
# query = "красная машина"
# query_embedding = get_text_embeddings([query])

# similarities = cosine_similarity(query_embedding, image_embeddings)[0]
# top_indices = np.argsort(similarities)[::-1][:5]  # Топ-5

# print(f"Топ-5 изображений для запроса '{query}':")
# for idx in top_indices:
#     print(f"{image_paths[idx].name}: {similarities[idx]:.4f}")